<a href="https://colab.research.google.com/github/solalcaraz/TP-MATEIII-RN/blob/main/TP_Mate_RNN_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import zipfile as zf
#files = zf.ZipFile("data/maestro-v3.0.0-midi.zip", 'r')
#files.extractall('data/')
#files.close()
!pip install pretty_midi
!pip install tensorflow

instalo todos los paquetes relevantes para procesar y escuchar archivos MIDI, glob para iterar sobre los archivos del dataset, collections para el diccionario con valor por defecto, y pathlib para abrir el archivo.

In [ ]:
import collections
import glob
import numpy as np
import pathlib
import pandas as pd
import pretty_midi
import tensorflow as tf


utilizamos tres variables para definir una "nota", tono (pitch), step, y duración.
tono es la calidad sonora de la nota, lo que la distingue de otras.
step es el tiempo entre el inicio de una nota y el inicio de la previa
duración lo que dura en milisegundos cada nota.

In [ ]:
#directorio = pathlib.Path('data/maestro-v3.0.0')
#filenames = glob.glob(str(data_dir/'**/*.mid*'))

sample_file = "/content/MIDI-Unprocessed_01_R1_2008_01-04_ORIG_MID--AUDIO_01_R1_2008_wav--3.midi"
print(sample_file)

def midi_a_notas(midifile: str) -> pd.DataFrame:
    pm = pretty_midi.PrettyMIDI(midifile)
    instrumento = pm.instruments[0]
    notas = collections.defaultdict(list)

    notas_orden = sorted(instrumento.notes, key=lambda nota: nota.start)
    previa_inicio = notas_orden[0].start

    for nota in notas_orden:
        inicio = nota.start
        fin = nota.end
        notas['pitch'].append(nota.pitch)
        notas['start'].append(inicio)
        notas['end'].append(fin)
        notas['step'].append(inicio - previa_inicio)
        notas['duration'].append(fin - inicio)
        previa_inicio = inicio

    return pd.DataFrame({nombre: np.array(valor) for nombre, valor in notas.items()})


/content/MIDI-Unprocessed_01_R1_2008_01-04_ORIG_MID--AUDIO_01_R1_2008_wav--3.midi


Codigo robado: escribe un dataframe con notas en un archivo midi y lo devuelve como prettyMIDI para poder escucharlo:

In [ ]:
def notes_to_midi(
  notes: pd.DataFrame,
  out_file: str,
  instrument_name: str,
  velocity: int = 100,  # note loudness
) -> pretty_midi.PrettyMIDI:

  pm = pretty_midi.PrettyMIDI()
  instrument = pretty_midi.Instrument(
      program=pretty_midi.instrument_name_to_program(
          instrument_name))

  prev_start = 0
  for i, note in notes.iterrows():
    start = float(prev_start + note['step'])
    end = float(start + note['duration'])
    note = pretty_midi.Note(
        velocity=velocity,
        pitch=int(note['pitch']),
        start=start,
        end=end,
    )
    instrument.notes.append(note)
    prev_start = start

  pm.instruments.append(instrument)
  pm.write(out_file)
  return pm

In [ ]:
notasGlobal = []
tam_muestra = 10
#for f in filenames[:tam_muestra]:
notas = midi_a_notas(sample_file)
    #notasGlobal.append(notas)
#notasGlobal = pd.concat(notasGlobal)

indice_notas = ['pitch', 'step', 'duration']
notas_entrenamiento = np.stack([notas[key] for key in indice_notas], axis=1)
np.set_printoptions(precision=3)
dataset_nota = tf.data.Dataset.from_tensor_slices(notas_entrenamiento)
print(notas_entrenamiento)
print(list(dataset_nota.as_numpy_iterator()))

[[69.     0.     1.025]
 [61.     0.807  0.771]
 [69.     0.658  0.07 ]
 ...
 [44.     0.026  0.24 ]
 [61.     1.366  1.361]
 [53.     0.033  1.398]]
[array([69.   ,  0.   ,  1.025]), array([61.   ,  0.807,  0.771]), array([69.   ,  0.658,  0.07 ]), array([69.   ,  0.254,  2.234]), array([61.   ,  0.021,  0.371]), array([64.   ,  0.004,  0.383]), array([64.   ,  0.878,  0.316]), array([61.   ,  0.052,  0.266]), array([64.   ,  0.842,  0.543]), array([61.   ,  0.03 ,  0.523]), array([68.   ,  0.434,  0.421]), array([71.   ,  0.428,  0.544]), array([64.   ,  0.022,  0.574]), array([61.   ,  0.01 ,  0.522]), array([69.   ,  0.417,  0.191]), array([69.   ,  0.47 ,  0.285]), array([63.   ,  0.022,  0.368]), array([60.   ,  0.005,  0.37 ]), array([44.   ,  0.724,  0.055]), array([63.   ,  0.464,  0.449]), array([51.   ,  0.014,  0.085]), array([52.   ,  0.427,  0.039]), array([68.   ,  0.177,  0.076]), array([63.   ,  0.298,  0.618]), array([68.   ,  0.001,  3.337]), array([60.   ,  0.004,  

In [ ]:
def ventanizar(dataset, tam_ventana, alfabeto_notas = 128,) -> tf.data.Dataset:
  tam_ventana = tam_ventana+1

  windows = dataset.window(tam_ventana, shift=1, stride=1,
                              drop_remainder=True)

  flatten = lambda x: x.batch(tam_ventana, drop_remainder=True)
  sequences = windows.flat_map(flatten)

  # Normalize tono de las notas
  def normalizar_tono(x):
    x = x/[tam_ventana,1.0,1.0]
    return x

  # Partimos los indices
  def partir_indices(sequences):
    inputs = sequences[:-1]
    indices_todos = sequences[-1]
    indices = {key:indices_todos[i] for i,key in enumerate(indice_notas)}

    return normalizar_tono(inputs), indices

  return sequences.map(partir_indices, num_parallel_calls=tf.data.AUTOTUNE)